# Sequence Models & How to use them

**학습목표**
1. Bidirectional layer를 이해한다.
2. Bidirectional layer를 사용할 줄 안다.

-----------------
* Thanks to : 한기영 대표님 @ Data Insight

## 1.환경 및 데이터 준비

### Import Packages

In [33]:
#라이브러리들을 불러오자.
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Flatten, MaxPool1D, Conv1D, LSTM, GRU, Bidirectional
from tensorflow.keras.models import Model



### Data Loading

In [34]:
url = 'https://raw.githubusercontent.com/RayleighKim/Example_datasets/master/Stock_Edwards_Lifesciences_corporation.csv'

# 판다스로 데이터를 불러오시오.
data = pd.read_csv(url)

##3.데이터 준비

### Date 컬럼을 제거하시오.

In [35]:
data = data.drop('Date', axis=1)

## 4.Sequence 데이터 구조로 만들기

**조건**
* 내일의 Close를 예측할 것이다. 이를 Y로 둘 것.
* timestep은 10주를 본다. (주식은 5일이 1주일)
* 맞추어 전처리

**데이터 분할 규칙**
* 가장 최근 1주일을 테스트 데이터로 둔다.
* 테스트 데이터를 제외하고, 가장 최근 2주를 벨리데이션 데이터로 둔다.

In [36]:
# 판다스 데이터 프레임을 넘파이로 옮김.
data_arr = data.values

In [37]:
# 여기에 의미있는 기간(timestep을 지정해 봅시다.)
timestep = 20
data_len = len(data_arr)
x = np.array([data_arr[ i : i+timestep, : ] for i in range( data_len - timestep )])
y = np.array([data_arr[ i+timestep, 3 ] for i in range( data_len - timestep )])

x_train, x_val, x_test = x[:-15], x[-15:-5], x[-5 : ]
y_train, y_val, y_test = y[:-15], y[-15:-5], y[-5 : ]


## 모양 확인 필수
print('[ # , timestep, feature수 ] <-- 데이터의 구조 : ')
print(x.shape, y.shape)
print('-------------------------------')
print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)
print(x_test.shape, y_test.shape)
print('-------------------------------')

[ # , timestep, feature수 ] <-- 데이터의 구조 : 
(4372, 20, 6) (4372,)
-------------------------------
(4357, 20, 6) (4357,)
(10, 20, 6) (10,)
(5, 20, 6) (5,)
-------------------------------


데이터의 구조 : [n, timestep, feature수]

# 같이 해보기
**[참고링크](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Bidirectional)**
1. 적절한 인풋 레이어를 구성할 것
2. Conv1D
    * 16개 필터 사용할 것
    * window_size(filter_size)는 5일
    * activation은 swish
3. MaxPool1D
    * 필터 사이즈(window size)는 2일
4. Bidirectional 레이어
    * LSTM, 히든스테이트 노드 32개
5. Bidirectional 레이어
    * LSTM, 히든스테이트 노드 32개
4. 플래튼
5. 회귀를 위한 적절한 아웃풋 레이어



In [38]:
x_train.shape, y_train.shape

((4357, 20, 6), (4357,))

In [41]:
"""
1. 적절한 인풋 레이어를 구성할 것
2. Conv1D
    * 16개 필터 사용할 것
    * window_size(filter_size)는 5일
    * activation은 swish
3. MaxPool1D
    * 필터 사이즈(window size)는 2일
4. Bidirectional 레이어
    * LSTM, 히든스테이트 노드 32개
5. Bidirectional 레이어
    * LSTM, 히든스테이트 노드 32개
4. 플래튼
5. 회귀를 위한 적절한 아웃풋 레이어
"""

# 1. 세션 클리어
tf.keras.backend.clear_session()
# 2. 레이어 연결
il = Input(shape=(20, 6))
hl = Conv1D(filters=16, 
            kernel_size=5, 
            strides=1, 
            padding='valid', 
            activation='swish')(il)
hl = MaxPool1D(pool_size=2)(hl)
hl = Bidirectional( layer=LSTM(32, return_sequences=True), 
                   name='Bidirect_LSTM_1' )(hl)
hl = Bidirectional( layer=LSTM(32, return_sequences=True), 
                   name='Bidirect_LSTM_2' )(hl)
hl = Flatten()(hl)
ol = Dense(1, activation='relu')(hl) # 음수를 0으로 처리
# 3. 모델 선언
model = Model(il, ol)
# 4. 컴파일
model.compile(loss='mae', optimizer='adam')
# 요약
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 20, 6)]           0         
                                                                 
 conv1d (Conv1D)             (None, 16, 16)            496       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 8, 16)            0         
 )                                                               
                                                                 
 Bidirect_LSTM_1 (Bidirectio  (None, 8, 64)            12544     
 nal)                                                            
                                                                 
 Bidirect_LSTM_2 (Bidirectio  (None, 8, 64)            24832     
 nal)                                                            
                                                             

In [42]:
model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=30)

Epoch 1/30
137/137 [==============================] - 15s 44ms/step - loss: 22.5799 - val_loss: 100.2691
Epoch 2/30
137/137 [==============================] - 3s 24ms/step - loss: 21.3700 - val_loss: 98.1350
Epoch 3/30
137/137 [==============================] - 3s 21ms/step - loss: 21.3356 - val_loss: 98.7225
Epoch 4/30
127/137 [==========================>...] - ETA: 0s - loss: 21.2492

KeyboardInterrupt: ignored

In [26]:
from sklearn.metrics import *
y_pred = model.predict(x_test)
mean_squared_error(y_test, y_pred)**0.5

1/1 [==============================] - 0s 52ms/step


1599819.9230546847

## Q1. Conv1D + Bidirectional

**[공식문서 적극 참고](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Bidirectional)**
Bidirectional)**
1. 적절한 인풋 레이어를 구성할 것
2. Conv1D
    * 32개 필터 사용할 것
    * window_size(filter_size)는 10일
    * activation은 swish
3. MaxPool1D
    * 필터 사이즈(window size)는 2일
4. Bidirectional 레이어
    * forward layer : LSTM, 히든스테이트 노드 24개
    * backward layer : GRU, 히든스테이트 노드 16개
4. Bidirectional 레이어
    * forward layer : LSTM, 히든스테이트 노드 24개
    * backward layer : GRU, 히든스테이트 노드 24개
4. 플래튼
6. Fully Connected : 노드 256개, swish
5. 회귀를 위한 적절한 아웃풋 레이어


In [ ]:
####################
## Your Code Here ##
####################



In [ ]:
## 학습도 시킬 것



In [ ]:
# Q1. 테스트 셋에서의 RMSE를 출력하여라.

